In [1]:
import pandas as pd
import numpy as np
from HFC32_emimTf2N_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.013
    init_kappa_1_2A = -0.022
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['emimTf2N'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['emimTf2N','emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_A['emimTf2N', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'emimTf2N'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['emimTf2N', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_B['emimTf2N', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_C['emimTf2N', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_C['emimTf2N', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'emimTf2N'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['emimTf2N'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'emimTf2N'].fix(float(data['x_emimTf2N']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['emimTf2N'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['emimTf2N', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['emimTf2N', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'emimTf2N'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'emimTf2N'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['emimTf2N', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'emimTf2N']"]


In [3]:
data = pd.read_csv('R32_emimTF2N.csv')

print(data)

    temperature  pressure  x_R32  x_emimTf2N
0        283.15     10200  0.014       0.986
1        283.15    100100  0.136       0.864
2        283.15    250300  0.306       0.694
3        283.15    399300  0.448       0.552
4        283.15    549300  0.570       0.430
5        283.15    700400  0.672       0.328
6        283.15    849400  0.786       0.214
7        298.15      9600  0.010       0.990
8        298.15     99800  0.095       0.905
9        298.15    250200  0.218       0.782
10       298.15    399400  0.325       0.675
11       298.15    549400  0.417       0.583
12       298.15    699500  0.499       0.501
13       298.15    849500  0.570       0.430
14       298.15    999700  0.643       0.357
15       323.15     10200  0.005       0.995
16       323.15    100300  0.055       0.945
17       323.15    249700  0.133       0.867
18       323.15    400100  0.200       0.800
19       323.15    549500  0.264       0.736
20       323.15    700300  0.320       0.680
21       3

In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_emimTf2N"] += data_["x_emimTf2N"] * noise_x
    data_.loc[data_['x_emimTf2N'] <= 0.0001, 'x_emimTf2N'] = 0.0001
    data_.loc[data_['x_emimTf2N'] >= 0.9999 , 'x_emimTf2N'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.01616047379572957
2022-03-07 16:24:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:24:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07

2022-03-07 16:25:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:36 [INFO] idaes

2022-03-07 16:26:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:27:05 [INFO] idaes

2022-03-07 16:28:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:29 [INFO] idaes

2022-03-07 16:30:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:30:16 [INFO] idaes

...

2022-03-07 17:23:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:23:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:23:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:23:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:23:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[emimTf2N,R32] = -0.02194700258205105
fs.properties.PR_kappa_A[R32,emimTf2N] = -0.024994418216684522
91
0.009508290240636862
2022-03-07 17:23:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:23:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022

2022-03-07 17:24:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:24:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:24:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[emimTf2N,R32] = -0.027441033215287368
fs.properties.PR_kappa_A[R32,emimTf2N] = -0.025080910830361177
93
0.11650612946937583
2022-03-07 17:24:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:24:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:24:51 [INFO] idaes.init.fs.state_block: Pro

2022-03-07 17:25:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:25:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:25:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[emimTf2N,R32] = -0.02958579446645239
fs.properties.PR_kappa_A[R32,emimTf2N] = -0.0251098333239949
95
0.08649109884161596
2022-03-07 17:26:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:26:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:26:06 [INFO] idaes.init.fs.state_block: Proper

2022-03-07 17:27:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:27:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:27:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[emimTf2N,R32] = -0.02410765129484529
fs.properties.PR_kappa_A[R32,emimTf2N] = -0.025037008545378273
97
0.04849661027658456
2022-03-07 17:27:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:27:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:27:20 [INFO] idaes.init.fs.state_block: Prop

2022-03-07 17:28:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:28:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:28:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[emimTf2N,R32] = -0.020618975316098912
fs.properties.PR_kappa_A[R32,emimTf2N] = -0.02498484007404442
99
-0.028150436951320457
2022-03-07 17:28:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:28:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:28:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:28:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
202

In [5]:
print(results_params)

[[-0.0281707  -0.0250908 ]
 [-0.02074356 -0.02498562]
 [-0.02271593 -0.0250015 ]
 [-0.02396986 -0.02503358]
 [-0.02081901 -0.02498611]
 [-0.02079474 -0.02498595]
 [-0.03266864 -0.02514979]
 [-0.0274501  -0.02508105]
 [-0.01802924 -0.0249578 ]
 [-0.02067041 -0.02498516]
 [-0.02074098 -0.02498561]
 [-0.02420224 -0.02503824]
 [-0.02980023 -0.0251125 ]
 [-0.02081241 -0.02498606]
 [-0.02141442 -0.02499023]
 [-0.02043989 -0.02498375]
 [-0.02629721 -0.02506554]
 [-0.02056594 -0.02498451]
 [-0.02209628 -0.02499569]
 [-0.02287097 -0.02502089]
 [-0.02290302 -0.02500343]
 [-0.02087054 -0.02498644]
 [-0.02225191 -0.02499707]
 [-0.03943755 -0.02523678]
 [-0.03182741 -0.02513761]
 [-0.02042891 -0.02498369]
 [-0.03012837 -0.02511691]
 [-0.02080833 -0.02498604]
 [-0.02843208 -0.02509336]
 [-0.02166436 -0.02499213]
 [-0.02079818 -0.02498597]
 [-0.02848118 -0.025094  ]
 [-0.03167689 -0.02513718]
 [-0.02065187 -0.02498504]
 [-0.02134846 -0.02498974]
 [-0.03128446 -0.02513197]
 [-0.02028788 -0.02498284]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_emimTf2N_10_2.csv')